# Dynamic Attribution and Properties

## Data Wrangling with Dynamic Attributes

In [1]:
import json

with open('data/osconfeed.json') as fp:
    feed = json.load(fp)

In [2]:
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [3]:
for key, value in sorted(feed['Schedule'].items()):
    print(f'{len(value): 3}\t{key}')

  1	conferences
 484	events
 357	speakers
 53	venues


In [4]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

---

In [5]:
import keyword
from collections import abc


class FrozenJSON:
    """ A read-only facade for navigating a JSON-like object
        using attribute natation
    """

    # As a class method, the first argument `__new__` gets is the class itself
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            # The default behavior is to delegate to the `__new__` of a superclass. 
            return super().__new__(cls)
        # The remaining lines of `__new__` are exactly as in the old `build` method. 
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value
    
    def __getattr__(self, name):
        try:
            return getattr(self.__data, name)
        except AttributeError:
        # Now we just call the `FrozenJSON` class, which Python handles by calling `FrozenJSON.__new__`. 
            return FrozenJSON.build(self.__data[name])
    
    def __dir__(self):
        return self.__data.keys()
    
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        return obj

In [6]:
raw_feed = json.load(open('data/osconfeed.json'))

feed_json = FrozenJSON(raw_feed)
feed_json.keys()

dict_keys(['Schedule'])

In [7]:
len(feed_json.Schedule.speakers)

357

In [8]:
talk = feed_json.Schedule.events[40]
talk

In [9]:
talk.name

'There *Will* Be Bugs'

In [10]:
try:
    print(talk.flavor)
except KeyError:
    print(f'Trying to read a missing attribute. ')

Trying to read a missing attribute. 


## Computed Properties

In [11]:
import json
import inspect

JSON_PATH = 'data/osconfeed.json'


class Record:

    __index = None      # private class attribute will eventually hold a reference to the `dict` returned by `load`. 

    def __init__(self, **kwargs) -> None:
        self.__dict__.update(kwargs)
    
    def __repr__(self):
        return f'<{self.__class__.__name__} serial={self.serial!r}>'
    
    @staticmethod
    def fetch(key):
        if Record.__index is None:
            Record.__index = load()
        # Use it to retrieve the record with the given `key`
        return Record.__index[key]


def load(path=JSON_PATH):
    records = {}
    with open(path) as fp:
        raw_data = json.load(fp)
    for collection, raw_records in raw_data['Schedule'].items():
        record_type = collection[:-1]
        # Capitalize the `record_type` to get a possible class;
        # e.g., `event` becomes `Event`. 
        cls_name = record_type.capitalize()
        # Get an object by that name from the module global scope;
        # get the `Record` class if there's no such object. 
        cls = globals().get(cls_name, Record)
        if inspect.isclass(cls) and issubclass(cls, Record):
            factory = cls
        else:
            factory = Record
        for raw_record in raw_records:
            key = f'{record_type}.{raw_record["serial"]}'
            records[key] = factory(**raw_record)
    return records

In [12]:
records = load()
speaker = records['speaker.3471']
speaker

<Record serial=3471>

In [13]:
speaker.name, speaker.twitter

('Anna Martelli Ravenscroft', 'annaraven')

---

In [14]:
from functools import cache

class Event(Record):

    def __repr__(self):
        try:
            return f'<{self.__class__.__name__} {self.name!r}>'
        except AttributeError:
            return super().__repr__()
    
    # The `venue` property builds a key from the `venue_serial` attribute, and passes it to the `fetch` class method, 
    # inherited from `Record` (the reason for using `self.__class__` is explained shortly). 
    @property
    @cache
    def venue(self):
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key)
    
    # `@cache_property` will overide
    @property
    @cache
    def speakers(self):
        # The data we want is in a `speakers` attribute, but we must retrive it directly from 
        # the instance `__dict__` to avoid a recursive call to the `speakers` propertiy. 
        spkr_serials = self.__dict__['speakers']  
        fetch = self.__class__.fetch
        return [fetch(f'speaker.{key}')
                for key in spkr_serials]

Inside the `speakers` method, trying to read `self.speakers` will invoke the property itself, quickly raising a `RecursionError`. 

However, if we read the same data via `self.__dict__['speakers']`, Python's usual algorithm for retrieving attributes is bypassed, the property is not called, and the recursion is avoided. 

For this reason, reading or writing data directly to an object's `__dict__` is a common Python metaprogramming trick. 

In [15]:
event = Record.fetch('event.33950')
event

<Event 'There *Will* Be Bugs'>

In [16]:
print(event.venue, event.venue_serial)

<Record serial=1449> 1449


## Using a Property for Attribute Validation

In [17]:
class LineItem:

    data = 'the class data attr'

    def __init__(self, description, weight, price):
        self.descripton = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    # `@property` decorates the getter method. 
    @property
    def weight(self):
        return self.__weight

    # The decorated getter has a `.setter` attribute, which is also a decorator;
    # this ties the getter and setter together. 
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError(f'value: {value} must be > 0')
    
    @property
    def prop(self):
        return 'the prop value'


In [18]:
a = LineItem('as', 20, 1)
# a.weight = -20

In [19]:
vars(a)

{'descripton': 'as', '_LineItem__weight': 20, 'price': 1}

The full signature of the `property` constructor:

```python
property(fget=None, fset=None, fdel=None, doc=None)
```


`vars` returns the `__dict__` of `obj`

In [20]:
LineItem.prop

In [21]:
print(a.prop)

try:
    a.prop = 'foo'
except AttributeError:
    print('Opps')

the prop value
Opps


Reading `a.prop` executes the property getter. 

But Trying to set an instance `prop` attribute fails. 

In [22]:
a.__dict__['prop'] = 'foo'
print(vars(a))
print(a.prop)

{'descripton': 'as', '_LineItem__weight': 20, 'price': 1, 'prop': 'foo'}
the prop value


Putting `prop` directly in the `a.__dict__` works. And we can see that `a` then has two instance attribute: `data` and `prop`. 

However, reading `a.prop` still runs the property getter. The property is not shadowed by an instance attribute. 

In [23]:
LineItem.prop = 'baz'

a.prop

'foo'

Overwriting `LineItem.prop` destroys the property object. 

Then `LineItem.prop` retrives the instance attribute. `LineItem.prop` is not a property is not more, so it no longer overrides `a.prop`. 

---

### Properties Override Instance Attributes

In [24]:
a.data = 'bar'
print(a.data)
print(LineItem.data)

LineItem.data = property(lambda self: 'the "data" prop value')
print(a.data, LineItem.data, sep='\n')

bar
the class data attr
the "data" prop value


overwrite `LineItem.data` with a new property. `a.data` then is shadowed by the `LineItem.data` property. 

In [25]:
del LineItem.data
a.data

'bar'

Delete the property. Then `a.data` reads the instance `data` attribute again. 

## Handing Attribute Deletion

We can use the `del` statement to delete not only variables, but also attributes. 

In [26]:
class BlackKnight:

    def __init__(self):
        self.phrases = [
            ('an arm', "'Tis but a scratch."),
            ('another arm', "It's just a flesh wound."),
            ('a leg', "I'm invincible!"),
            ('another leg', "All right, we'll call it a draw.")
        ]

    @property
    def member(self):
        print('next member is:')
        return self.phrases[0][0]

    @member.deleter
    def member(self):
        member, text = self.phrases.pop(0)
        print(f'BLACK KNIGHT (loses {member}) -- {text}')


In [27]:
knight = BlackKnight()
knight.member

next member is:


'an arm'

In [28]:
del knight.member

BLACK KNIGHT (loses an arm) -- 'Tis but a scratch.


By the way, if you are not using a property, attribute deletion can also be handled by implementing the lower-level `__delattr__` special method. 